## UI Tests

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

from time import sleep
import datetime as DT

In [ ]:
browser = webdriver.Chrome(executable_path="C:/chromedriver")

## Generic functions used in test 1 and 3

In [ ]:
# Location
def setLocation(location):
    input_location = browser.find_element_by_id('bigsearch-query-detached-query')
    input_location.clear()
    input_location.send_keys(location)

    sleep(1)
    # browser.find_element_by_id('bigsearch-query-detached-query').get_attribute('value')

In [ ]:
# Check-In
def setCheckInOut(in_date, out_date):
    checkin_btn = browser.find_element_by_css_selector('[data-testid="structured-search-input-field-split-dates-0"]')
    checkin_btn.click()

    sleep(3)

    checkin_date = browser.find_element_by_css_selector('[data-testid="datepicker-day-' + str(in_date) + '"]')
    checkin_date.click()

    checkout_date = browser.find_element_by_css_selector('[data-testid="datepicker-day-' + str(out_date) + '"]')
    checkout_date.click()

In [ ]:
# Guests
def setGuests(adults, children):
    browser.find_element_by_css_selector('[data-testid="structured-search-input-field-guests-button"]').click()

    adults_plus_btn = browser.find_element_by_css_selector('[data-testid="stepper-adults-increase-button"]')
    
    for i in range(adults):
        adults_plus_btn.click()

    child_plus_btn = browser.find_element_by_css_selector('[data-testid="stepper-children-increase-button"]')
    
    for j in range(children):
        child_plus_btn.click()

In [ ]:
# Click Search Button
def search():
    search_btn = browser.find_element_by_css_selector('[data-testid="structured-search-input-search-button"]')
    search_btn.click()

In [ ]:
# Verify filters
def verify_filters(location_city, nbr_guests, in_date, out_date):
    # Location
    filter_location = browser.find_element_by_xpath("//button[@data-index='0']/div[@class='_1g5ss3l']")
    if (filter_location.text != location_city):
        print('Location is incorrect')

    # Date
    filter_date = browser.find_element_by_xpath("//button[@data-index='1']/div[@class='_1g5ss3l']")
    if (in_date.month == out_date.month):
        str_date = in_date.strftime("%b") + ' ' + str(in_date.day) + ' – ' + str(out_date.day)
    else: 
        str_date = in_date.strftime("%b") + ' ' + str(in_date.day) + ' – ' + in_date.strftime("%b") + ' ' + str(out_date.day)

    if (filter_date.text != str_date):
        print('Dates are incorrect')

    # Guests
    filter_guests = browser.find_element_by_xpath("//button[@data-index='2']/div[@class='_1g5ss3l']")
    if (int(filter_guests.text.split(' · ')[0].replace(' guests', '')) != nbr_guests):
        print('Number of guests is incorrect')

In [ ]:
# Verify that the properties displayed on the first page can accommodate at least the selected number of guests
def verify_guests(nbr_guests):
    rooms = browser.find_elements_by_xpath("//div[@class='_kqh46o']")
    i = 0
    for room in rooms:
        if i%2 == 0:
            if (int(room.text.split(' · ')[0].replace(' guests', '')) < nbr_guests):
                print('Number of guests is incorrect')
        i = i+1

### End of functions

## Test 1: Verify that the results match the search criteria

In [ ]:
browser.get('https://www.airbnb.com/')
sleep(3)

location = 'Rome, Italy'
location_city = 'Rome'
adults = 2
children = 1

today = DT.date.today()
in_date = today + DT.timedelta(days=7)
out_date = in_date + DT.timedelta(days=7)
    
setLocation(location)
setCheckInOut(in_date, out_date)
setGuests(adults, children)

search()
sleep(4)

verify_filters(location_city, adults+children, in_date, out_date)
verify_guests(adults+children)

## Test 2: Verify that the results and details page match the extra filters

In [ ]:
# Add extra filters : Bedrooms + Pool

# The problem here is that the buttons depends on the screen size
# Sometimes the add bedrooms buttons is next the More filters button
# Sometimes it is in the More Filter Popup

more_filters_btn = browser.find_element_by_xpath("//div[@id='menuItemButton-dynamicMoreFilters']/button[@class='_t6p96s']")

try:
    bedrooms_btn = browser.find_elements_by_xpath("//div[@id='menuItemButton-rooms_and_beds']/button[@class='_t6p96s']")
except NoSuchElementException:
    pass

if (len(bedrooms_btn) > 0):
    bedrooms_btn = bedrooms_btn[0]
    bedrooms_btn.click()
    
    add_bedrooms = browser.find_element_by_css_selector('[data-testid="filterItem-rooms_and_beds-stepper-min_bedrooms-0-increase-button"]')

    for i in range(5):
        add_bedrooms.click()

    more_filters_btn.click()

else:
    more_filters_btn.click()
    
    add_bedrooms = browser.find_element_by_css_selector('[data-testid="filterItem-rooms_and_beds-stepper-min_bedrooms-0-increase-button"]')
    
    for i in range(5):
        add_bedrooms.click()

sleep(2)
pool_btn = browser.find_element_by_css_selector('[name="Pool"]').click()

browser.find_element_by_css_selector('[data-testid="more-filters-modal-submit-button"]').click()

sleep(3)

In [ ]:
# Verify that the properties displayed on the first page have at least the number of selected bedroom.
rooms = browser.find_elements_by_xpath("//div[@class='_kqh46o']")
i = 0
for room in rooms:
    if i%2 == 0:
        nbr_bedrooms = room.text.split(' · ')[1].replace(' bedrooms', '')
        if (int(nbr_bedrooms) < 5):
            print(nbr_bedrooms)
            print('Number of bedrooms is incorrect')
    i = i+1

In [ ]:
# Open the details of the first property.
browser.find_elements_by_xpath("//div[@class='_8s3ctt']")[0].click()
sleep(3)

#get current window handle
p = browser.current_window_handle

#get child windows
chwd = browser.window_handles
for w in chwd:
    #switch focus to child window
    if(w!=p):
        browser.switch_to.window(w)
        break

sleep(4)

In [ ]:
# Couldn't find facilites section -- It's Amenities
amenities_div = browser.find_element_by_xpath("//div[@data-section-id='AMENITIES_DEFAULT']")
pool = amenities_div.find_elements_by_xpath("//*[contains(text(), 'Pool')]")

if (len(pool) == 0):
    print("Pool not found")

## Test 3: Verify that a property is displayed on the map correctly

In [ ]:
browser.get('https://www.airbnb.com/')
sleep(3)

location = 'Rome, Italy'
location_city = 'Rome'
adults = 2
children = 1

today = DT.date.today()
in_date = today + DT.timedelta(days=7)
out_date = in_date + DT.timedelta(days=7)
    
setLocation(location)
setCheckInOut(in_date, out_date)
setGuests(adults, children)

search()
sleep(4)

In [ ]:
from selenium.webdriver.common.action_chains import ActionChains

first_element = browser.find_element_by_xpath("//div[@class='_8s3ctt']")
first_element_title = first_element.find_elements_by_tag_name("a")[0].get_attribute('aria-label')
first_element_price = first_element.find_elements_by_class_name('_1p7iugi')[0].text

print(first_element_price)
# get the price : sometimes the discounted price is added, we should take that into consideration
price = first_element_price.replace('Price:', '')
price = price.replace('Previous price:', '')

price = price.split('Discounted price:', 1)

if (len(price) > 1):
    price = price[1].strip()
else:
    price = price[0].strip()

print(price)

hover = ActionChains(browser).move_to_element(first_element)
hover.perform()

sleep(2)

# Because there's no ids, no classes, nothing specific to differentiate the price tags on the map, 
# I had to use the title and price defined in aria-label of the price tag button
# button > div > div_color
on_map_tag = browser.find_element_by_xpath("//button[@aria-label=' " + first_element_title + " " + price + "']")
div_color = on_map_tag.find_elements_by_tag_name('div')[1] #the second div is the one that changes style 

if (div_color.value_of_css_property('background-color') != 'rgba(34, 34, 34, 1)'):
    print("Price on map did not change color on hover")
    
on_map_tag.click()

In [ ]:
try:
    popup_title = browser.find_elements_by_xpath("//div[@class='_5y5rgc']/a[@aria-label='" + first_element_title  + "']")
except NoSuchElementException:
    print("Title in details popup is not identical to the search result")